In [3]:
import pandas as pd
import qgridnext as qgrid
from IPython.display import display, clear_output
import ipywidgets as widgets

class MultiIndexDataFrameToQGrid():
    def __init__(self, df=None):
        self.df = df
        self.combined_df = None
        self.df_cleaned = None
        self.qgrid_widget = None
        self.output_area = widgets.Output()

    def transpose_and_prepare_df(self):
        # Extract level values and create a DataFrame for each level
        levels = self.df.columns.names
        level_values = [self.df.columns.get_level_values(i) for i in range(len(levels))]

        # Create DataFrames for the levels and the values
        level_dfs = []
        for i, level_value in enumerate(level_values):
            level_df = pd.DataFrame([level_value], columns=self.df.columns)
            level_df.index = [levels[i]]
            level_dfs.append(level_df)

        value_df = pd.DataFrame(self.df.values, columns=self.df.columns, index=self.df.index)
        self.combined_df = pd.concat(level_dfs + [value_df]).T

        # Adjust column and row names to handle duplicates and maintain structure
        new_columns = [col if col not in levels else f'_{col}' for col in self.combined_df.columns]
        self.combined_df.columns = new_columns

        # Filter out columns with underscores (not needed for viewing)
        columns_to_keep = [col for col in self.combined_df.columns if '_' not in col]
        self.df_cleaned = self.combined_df[columns_to_keep]

    def display_in_qgrid(self):
        # Display the cleaned DataFrame in qgrid with toolbar enabled
        self.qgrid_widget = qgrid.show_grid(self.df_cleaned, show_toolbar=True)
        self.qgrid_widget.on('filter_changed', self.on_filter_change)
        display(self.qgrid_widget)
        display(self.output_area)

    def on_filter_change(self, event, widget):
        # Handle filtering changes by updating the output area
        with self.output_area:
            clear_output(wait=True)
            filtered_df = widget.get_changed_df().T
            display(filtered_df)
            
    def write_dataframe(self, df, file_path, delimiter=';'):
        if df is None:
            raise ValueError("Provided DataFrame is None, cannot serialize to file.")
        # Write DataFrame to CSV, ensuring MultiIndex is preserved
        df.to_csv(file_path, header=True, index=True, sep=delimiter)

    def read_dataframe(self, file_path, delimiter=';'):
        # Read DataFrame from CSV, handling MultiIndex headers and index
        df = pd.read_csv(file_path, header=[0, 1, 2], index_col=0, sep=delimiter)
        self.df = df
        return df

    def getWidgets(self):
        # Provide access to the qgrid widget and the output area for external manipulation
        return self.qgrid_widget, self.output_area

In [5]:
import pandas as pd
import numpy as np
import qgridnext as qgrid 
from IPython.display import display, clear_output
import ipywidgets as widgets

# Step 1: Create a DataFrame with Multi-Index Columns
data = np.random.randn(4, 8)

columns = pd.MultiIndex.from_product(
    [['Category1', 'Category2'], ['SubCat1', 'SubCat2'], ['Detail1', 'Detail2']],
    names=['Category', 'SubCategory', 'Detail']
)

df = pd.DataFrame(data, index=['Row1', 'Row2', 'Row3', 'Row4'], columns=columns)

csv_file_path = 'multiindex.csv'

midf = MultiIndexDataFrameToQGrid()
midf.write_dataframe(df, csv_file_path)
df = midf.read_dataframe(csv_file_path)

print("From CSV:", df)

midf.transpose_and_prepare_df()
midf.display_in_qgrid()

# # Display the original DataFrame
# print("Original DataFrame with Multi-Index Columns:")
# display(df)
# print("Original DataFrame Index:", df.index.tolist())
# print("Original DataFrame Columns:", df.columns.tolist())

# # Step 2: Convert Multi-Index Columns to Rows for qgrid
# # Extracting the level values and creating a DataFrame for each level
# levels = df.columns.names
# level_values = [df.columns.get_level_values(i) for i in range(len(levels))]

# # Create a DataFrame for the levels
# level_dfs = []
# for i, level_value in enumerate(level_values):
#     level_df = pd.DataFrame([level_value], columns=df.columns)
#     level_df.index = [levels[i]]
#     level_dfs.append(level_df)

# # Create a DataFrame for the values
# value_df = pd.DataFrame(data, columns=df.columns, index=df.index)

# # Concatenate the levels and values DataFrames
# combined_df = pd.concat(level_dfs + [value_df])

# # Transpose the DataFrame to fit qgrid format
# combined_df = combined_df.T

# # Print the transposed DataFrame's index and columns for clarity
# print("Transposed DataFrame Index:", combined_df.index.tolist())
# print("Transposed DataFrame Columns:", combined_df.columns.tolist())

# # Step 3: Ensure unique column names with modifications to avoid duplications
# # Adjusting column names to prepend "_" for MultiIndex names
# new_columns = []
# for i, col in enumerate(combined_df.columns):
#     if col in levels:  # Check if the column name is in the original MultiIndex names
#         new_columns.append(f'_{col}')  # Prepend "_" to the original name
#     else:
#         new_columns.append(f'{col}')  # Use the modified name with suffix for unique columns

# combined_df.columns = new_columns

# # Adjusting row names to append "_" for MultiIndex names
# # This is done during the creation of level_dfs
# level_dfs_adjusted = []
# for i, level_df in enumerate(level_dfs):
#     adjusted_index = [f'{index}_' for index in level_df.index]  # Append "_" to each index name
#     level_df.index = adjusted_index
#     level_dfs_adjusted.append(level_df)

# # Recreate the combined DataFrame with adjusted level DataFrames
# combined_df_adjusted = pd.concat(level_dfs_adjusted + [value_df]).T

# # Now, combined_df_adjusted has "_" prepended to column names that are part of the original MultiIndex
# # and "_" appended to row names that are part of the original MultiIndex

# # Display the DataFrame with separated levels
# print("Combined DataFrame for qgrid:")
# display(combined_df)
# print("Combined DataFrame Index after adjustments:", combined_df.index.tolist())
# print("Combined DataFrame Columns after adjustments:", combined_df.columns.tolist())


# # Step 1: Identify columns to keep (those without an underscore)
# columns_to_keep = [col for col in combined_df.columns if '_' not in col]

# # Step 2: Filter out unnecessary columns
# df_cleaned = combined_df[columns_to_keep]

# # Step 4: Create qgrid widget and output area for displaying filtered DataFrame
# qgrid_widget = qgrid.show_grid(df_cleaned, show_toolbar=True)
# output_area = widgets.Output()

# # Event handler to update the displayed DataFrame whenever changes are made in qgrid
# def on_filter_change(event, widget):
#     with output_area:
#         clear_output(wait=True)
#         filtered_df = widget.get_changed_df().T
#         display(filtered_df)

# # Attach the event handler to the qgrid widget
# qgrid_widget.on('filter_changed', on_filter_change)

# # Step 5: Display the qgrid widget and the output area
# display(qgrid_widget)
# display(output_area)

From CSV: Category    Category1                               Category2            \
SubCategory   SubCat1             SubCat2             SubCat1             
Detail        Detail1   Detail2   Detail1   Detail2   Detail1   Detail2   
Row1        -0.638659 -1.280511 -0.989833 -0.272823 -0.984178 -1.574066   
Row2        -0.284659 -0.396339  2.379253  0.428888  0.259878 -0.001103   
Row3        -1.698391  0.184663  1.560660 -0.874300  0.318672 -0.537066   
Row4         0.724209 -0.122690 -1.495077 -1.208416  0.826019 -0.266246   

Category                         
SubCategory   SubCat2            
Detail        Detail1   Detail2  
Row1         0.820901 -0.470299  
Row2         0.632874  0.786626  
Row3        -1.171358  1.170143  
Row4        -0.809362  0.150990  


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Output()

Original DataFrame with Multi-Index Columns:


Category    Category1                               Category2            \
SubCategory   SubCat1             SubCat2             SubCat1             
Detail        Detail1   Detail2   Detail1   Detail2   Detail1   Detail2   
Row1        -0.638659 -1.280511 -0.989833 -0.272823 -0.984178 -1.574066   
Row2        -0.284659 -0.396339  2.379253  0.428888  0.259878 -0.001103   
Row3        -1.698391  0.184663  1.560660 -0.874300  0.318672 -0.537066   
Row4         0.724209 -0.122690 -1.495077 -1.208416  0.826019 -0.266246   

Category                         
SubCategory   SubCat2            
Detail        Detail1   Detail2  
Row1         0.820901 -0.470299  
Row2         0.632874  0.786626  
Row3        -1.171358  1.170143  
Row4        -0.809362  0.150990

Original DataFrame Index: ['Row1', 'Row2', 'Row3', 'Row4']
Original DataFrame Columns: [('Category1', 'SubCat1', 'Detail1'), ('Category1', 'SubCat1', 'Detail2'), ('Category1', 'SubCat2', 'Detail1'), ('Category1', 'SubCat2', 'Detail2'), ('Category2', 'SubCat1', 'Detail1'), ('Category2', 'SubCat1', 'Detail2'), ('Category2', 'SubCat2', 'Detail1'), ('Category2', 'SubCat2', 'Detail2')]
Transposed DataFrame Index: [('Category1', 'SubCat1', 'Detail1'), ('Category1', 'SubCat1', 'Detail2'), ('Category1', 'SubCat2', 'Detail1'), ('Category1', 'SubCat2', 'Detail2'), ('Category2', 'SubCat1', 'Detail1'), ('Category2', 'SubCat1', 'Detail2'), ('Category2', 'SubCat2', 'Detail1'), ('Category2', 'SubCat2', 'Detail2')]
Transposed DataFrame Columns: ['Category', 'SubCategory', 'Detail', 'Row1', 'Row2', 'Row3', 'Row4']
Combined DataFrame for qgrid:


_Category _SubCategory  _Detail      Row1  \
Category  SubCategory Detail                                               
Category1 SubCat1     Detail1  Category1      SubCat1  Detail1 -0.638659   
                      Detail2  Category1      SubCat1  Detail2 -1.280511   
          SubCat2     Detail1  Category1      SubCat2  Detail1 -0.989833   
                      Detail2  Category1      SubCat2  Detail2 -0.272823   
Category2 SubCat1     Detail1  Category2      SubCat1  Detail1 -0.984178   
                      Detail2  Category2      SubCat1  Detail2 -1.574066   
          SubCat2     Detail1  Category2      SubCat2  Detail1  0.820901   
                      Detail2  Category2      SubCat2  Detail2 -0.470299   

                                   Row2      Row3      Row4  
Category  SubCategory Detail                                 
Category1 SubCat1     Detail1 -0.284659 -1.698391  0.724209  
                      Detail2 -0.396339  0.184663  -0.12269  
          SubCat2     Detail1  2.379253   1.56066 -1.495077  
                      Detail2  0.428888   -0.8743 -1.208416  
Category2 SubCat1     Detail1  0.259878  0.318672  0.826019  
                      Detail2 -0.001103 -0.537066 -0.266246  
          SubCat2     Detail1  0.632874 -1.171358 -0.809362  
                      Detail2  0.786626  1.170143   0.15099

Combined DataFrame Index after adjustments: [('Category1', 'SubCat1', 'Detail1'), ('Category1', 'SubCat1', 'Detail2'), ('Category1', 'SubCat2', 'Detail1'), ('Category1', 'SubCat2', 'Detail2'), ('Category2', 'SubCat1', 'Detail1'), ('Category2', 'SubCat1', 'Detail2'), ('Category2', 'SubCat2', 'Detail1'), ('Category2', 'SubCat2', 'Detail2')]
Combined DataFrame Columns after adjustments: ['_Category', '_SubCategory', '_Detail', 'Row1', 'Row2', 'Row3', 'Row4']


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Output()